In [ ]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
import os
import torch
from torch_geometric.data import Data

In [ ]:
data_dirs = ['twitter15', 'twitter16']

label_file_path = 'label.txt'
source_tweets_file_path = 'source_tweets.txt'
tree_file_dir = 'tree'
root_dir = '/content/drive/MyDrive/retweet_cascade'

num_features = 6

graph_list = []


In [ ]:
for data_dir in data_dirs:

    dir_path = os.path.join(root_dir, data_dir)

    for file_name in os.listdir(os.path.join(dir_path, tree_file_dir)):

        if not file_name.endswith('.txt'):
            continue

        tweet_id = file_name[:-4]

        edge_list = []
        with open(os.path.join(dir_path, tree_file_dir, file_name)) as f:
            for line in f:
                parent, child = line.strip().split('->')
                edge_list.append([int(parent.split(',')[1]), int(child.split(',')[1])])

        label = -1
        with open(os.path.join(dir_path, label_file_path)) as f:
            for line in f:
                if line.startswith('label:' + tweet_id):
                    label = int(line.strip().split(':')[1])
                    break

        features = torch.zeros((1, num_features))
        with open(os.path.join(dir_path, source_tweets_file_path)) as f:
            for line in f:
                if line.startswith(tweet_id):
                    cols = line.strip().split('\t')
                    features[0, 0] = int(cols[1])  # number of followers
                    features[0, 1] = int(cols[2])  # number of friends
                    features[0, 2] = float(cols[3])  # ratio of followers and friends
                    features[0, 3] = int(cols[4])  # number of history tweets
                    features[0, 4] = int(cols[5])  # registration time (year)
                    features[0, 5] = int(cols[6])  # whether verified account or not
                    break

        edge_index = torch.tensor(edge_list).t().contiguous()
        x = features.repeat(len(edge_list) + 1, 1)
        data = Data(x=x, edge_index=edge_index, y=label)

        graph_list.append(data)

In [ ]:

torch.save(graph_list, 'retweet_cascade_dataset.pt')